In [ ]:
import awscli
import sys
import selenium
import unittest
import boto3
import pandas as pd
import tabulate
import time
import requests
import csv
import socket 
import numpy as np
import dataframe
from bs4 import BeautifulSoup
from tabulate import tabulate
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
browser = webdriver.Chrome(r"C:\Users\...\chromedriver.exe")
browser.get('https://fortune.com/global500/2015/walmart')

df1 = pd.DataFrame(columns=['Revenues_$M','Profits_$M', 'Assets_$M', 'Employees', 'Profit_as_%_of_Revenues', 'Profits_as_%_of_Assets', 'Profits_as_%_of_Stockholder_Equity'])
c1 = []

for i in [x for x in range(500)]:       
    table = browser.find_element_by_xpath('//*[@id="content"]/div[5]/div[1]/table/tbody')    
    if i > 500:
        break


    try:
        print("Scraping Page no. " + str(i))
        i = i + 1
    
        for row in table.find_elements_by_css_selector('tr.dataTable__row--34F3j'):
            vals=[]
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[1]/table/tbody/tr[1]/td[2]')])).strip('[]').strip("''"))   
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[1]/table/tbody/tr[2]/td[2]')])).strip('[]').strip("''"))   
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[1]/table/tbody/tr[3]/td[2]')])).strip('[]').strip("''"))   
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[1]/table/tbody/tr[4]/td[2]')])).strip('[]').strip("''"))   
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[2]/table/tbody/tr[1]/td[2]')])).strip('[]').strip("''"))
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[2]/table/tbody/tr[2]/td[2]')])).strip('[]').strip("''"))
            vals.append(str(([cell.text for cell in row.find_elements_by_xpath('//*[@id="content"]/div[5]/div[2]/table/tbody/tr[3]/td[2]')])).strip('[]').strip("''"))            

        comp1 = str(([c.text for c in row.find_elements_by_xpath('//*[@id="content"]/div[1]/div[1]/div/h1/span')])).strip('[]').strip("''")    
                                                                           
        c1.append(comp1)
        
        s1 = pd.Series(vals,index=df1.columns)

        cs1 = pd.Series(c1) 
        df1 = df1.append(s1,ignore_index=True)
        
        try:

            WebDriverWait(browser, 6).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.singlePagination__next2--3D89W"))).click()
            time.sleep(5)

        except (socket.gaierror, requests.ConnectionError) as e:
            if e.errno != 10054:
                continue
            reconnect()



    except Exception as e:
        print(e)
        break

df1['company'] = cs1
df1['year'] = 2015

re_index = pd.Series(range(1,501))  
df1 = df1.set_index([re_index])    

print(df1)

browser.quit()


In [ ]:
df1.to_csv(r'C:\Users\...\key_financials_and_profit_ratio_all_2015_v1.csv')

In [ ]:
from botocore.client import Config
from io import StringIO

ACCESS_KEY_ID = 'your_access_key_id'
ACCESS_SECRET_KEY = 'your_access_secret_key'
BUCKET_NAME = 'ceo-culture-sagemaker'

csv_buffer =StringIO()
df1.to_csv(csv_buffer)

s3 = boto3.resource(
        's3',
        aws_access_key_id = ACCESS_KEY_ID,
        aws_secret_access_key=ACCESS_SECRET_KEY,
        config=Config(signature_version='s3v4')
        )
s3.Bucket(BUCKET_NAME).put_object(Key='key_financials_and_profit_ratio_all_2015_v1.csv', Body = csv_buffer.getvalue())

print("Uploaded")